<a href="https://colab.research.google.com/github/AdriiTrujillo/Fault_Tolerance_Colabs/blob/main/DFNN_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEEP FORWARD NEURAL NETWORK

Cuaderno para realizar la optimización, el entrenamiento y la evaluación de una DFNN.

In [ ]:
# Imports y permisos

%matplotlib inline
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import numpy as np
import re
import sys

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials

from keras.constraints import maxnorm
from keras.models import Sequential
from keras.models import load_model, Model
from keras.models import model_from_json
from keras.layers import Dense, Dropout, Activation, Input
from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from keras import applications
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [ ]:
# No creo que esto haga falta
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# OBTENCIÓN DE LA RED ÓPTIMA


# ENTRENAMIENTO

## RED ANTERIOR

En este apartado se procederá con el entrenamiento de una red con las mejores caracterisitcas obtenidas en el TFG de David Ruiz.

In [ ]:
# Obtención de todos los dataframes 
bubblesort_path = '/content/drive/Shareddrives/NN4FT_DB/Exp_Adrian/Data Frames/bubblesort-score.csv'
bubblesort_test_path = '/content/drive/Shareddrives/NN4FT_DB/Exp_Adrian/Data Frames/bubblesort-test.csv'

dijkstra_path = '/content/drive/Shareddrives/NN4FT_DB/Exp_Adrian/Data Frames/dijstra-score.csv'
dijkstra_test_path = '/content/drive/Shareddrives/NN4FT_DB/Exp_Adrian/Data Frames/dijstra-test.csv'

ndes_path = '/content/drive/Shareddrives/NN4FT_DB/Exp_Adrian/Data Frames/ndes-score.csv'
ndes_test_path = '/content/drive/Shareddrives/NN4FT_DB/Exp_Adrian/Data Frames/ndes-test.csv'

bubblesort_df = pd.read_csv(bubblesort_path, sep=";")
print(bubblesort_df.shape)
bubblesort_test_df = pd.read_csv(bubblesort_test_path, sep=";")
print(bubblesort_test_df.shape)

dijkstra_df = pd.read_csv(dijkstra_path, sep=";")
print(dijkstra_df.shape)
dijkstra_test_df = pd.read_csv(dijkstra_test_path, sep=";")
print(dijkstra_test_df.shape)


ndes_df = pd.read_csv(ndes_path, sep=";")
print(ndes_df.shape)
ndes_test_df = pd.read_csv(ndes_test_path, sep=";")
print(ndes_test_df.shape)

(686, 6)
(123, 6)
(92, 6)
(16, 6)
(592, 6)
(107, 6)


In [ ]:
# Obtención de los datos de entrenamiento y validación
bubblesort_Xdata = bubblesort_df[["time", "size"]].values
bubblesort_Ydata = bubblesort_df[["hang", "sdc"]].values

print(bubblesort_Xdata.shape)

dijkstra_Xdata = dijkstra_df[["time", "size"]].values
dijkstra_Ydata = dijkstra_df[["hang", "sdc"]].values

print(dijkstra_Xdata.shape)

ndes_Xdata = ndes_df[["time", "size"]].values
ndes_Ydata = ndes_df[["hang", "sdc"]].values

print(ndes_Xdata.shape)

x_data = np.concatenate((bubblesort_Xdata, dijkstra_Xdata, ndes_Xdata), axis=0)
y_data = np.concatenate((bubblesort_Ydata, dijkstra_Ydata, ndes_Ydata), axis=0)

y_data = y_data/100
scaler = preprocessing.StandardScaler() #Values with mean=0 and standard deviation=1
x_data = scaler.fit_transform(x_data)

print("x_data: ", x_data.shape)
print("y_data: ", y_data.shape)

(686, 2)
(92, 2)
(592, 2)
x_data:  (1370, 2)
y_data:  (1370, 2)


In [ ]:
# Obtención de los datos de test

bubblesort_test_Xdata = bubblesort_test_df[["time", "size"]].values
bubblesort_test_Ydata = bubblesort_test_df[["hang", "sdc"]].values

print(bubblesort_test_Xdata.shape)

dijkstra_test_Xdata = dijkstra_test_df[["time", "size"]].values
dijkstra_test_Ydata = dijkstra_test_df[["hang", "sdc"]].values

print(dijkstra_test_Xdata.shape)

ndes_test_Xdata = ndes_test_df[["time", "size"]].values
ndes_test_Ydata = ndes_test_df[["hang", "sdc"]].values

print(ndes_test_Xdata.shape)

x_test = np.concatenate((bubblesort_test_Xdata, dijkstra_test_Xdata, ndes_test_Xdata), axis=0)
y_test = np.concatenate((bubblesort_test_Ydata, dijkstra_test_Ydata, ndes_test_Ydata), axis=0)

y_test = y_test/100
scaler = preprocessing.StandardScaler() #Values with mean=0 and standard deviation=1
x_test = scaler.fit_transform(x_test)

print("x_test: ", x_test.shape)
print("y_test: ", y_test.shape)

(123, 2)
(16, 2)
(107, 2)
x_test:  (246, 2)
y_test:  (246, 2)


In [ ]:
#Fit your model
model = nn_model2()
model.fit(x=x_data, y=y_data, validation_data=(x_test, y_test), batch_size=16, epochs=400, verbose=2)
pesos = 'pesos_1.hdf5'
model.save_weights(pesos)

#Show predictions on test data
scores = model.evaluate(x_test, y_test, verbose=0)
print(scores)
# print(model.predict(x_test))

Epoch 1/400
86/86 - 2s - loss: 0.2706 - accuracy: 0.5358 - val_loss: 0.1085 - val_accuracy: 0.8618
Epoch 2/400
86/86 - 0s - loss: 0.1341 - accuracy: 0.6270 - val_loss: 0.0650 - val_accuracy: 0.7195
Epoch 3/400
86/86 - 0s - loss: 0.1107 - accuracy: 0.6555 - val_loss: 0.0706 - val_accuracy: 0.7195
Epoch 4/400
86/86 - 0s - loss: 0.0892 - accuracy: 0.7036 - val_loss: 0.0583 - val_accuracy: 0.7683
Epoch 5/400
86/86 - 0s - loss: 0.0773 - accuracy: 0.7161 - val_loss: 0.0494 - val_accuracy: 0.7398
Epoch 6/400
86/86 - 0s - loss: 0.0668 - accuracy: 0.7241 - val_loss: 0.0398 - val_accuracy: 0.7561
Epoch 7/400
86/86 - 0s - loss: 0.0612 - accuracy: 0.7139 - val_loss: 0.0413 - val_accuracy: 0.7195
Epoch 8/400
86/86 - 0s - loss: 0.0550 - accuracy: 0.7241 - val_loss: 0.0389 - val_accuracy: 0.7195
Epoch 9/400
86/86 - 0s - loss: 0.0487 - accuracy: 0.7350 - val_loss: 0.0370 - val_accuracy: 0.7520
Epoch 10/400
86/86 - 0s - loss: 0.0458 - accuracy: 0.7577 - val_loss: 0.0377 - val_accuracy: 0.7276
Epoch 11/

## GUARDAR LOS PESOS EN NN4FT_DB

Para guardar el archivo con los pesos de la red se mueve este a la carpeta de `Redes entrenadas`.

In [ ]:
import shutil
save_path = '/content/drive/Shareddrives/NN4FT_DB/Exp_Adrian/Redes Entrenadas'

ruta = shutil.move(pesos, save_path)

## RED NUEVA

# EVALUACIÓN

## RED ANTERIOR

In [ ]:
nn_path = '/content/drive/Shareddrives/NN4FT_DB/Exp_Adrian/Redes Entrenadas/pesos_1.hdf5'

# Crear y cargar la red neuronal
model = nn_model()
model.load_weights(nn_path)

# Para utilizar los datos de test (Comentar y descomentar)
# y_data = y_test
# x_data = x_test

# Correr la red nueronal
print('Corriendo la red ... ')
scores = model.evaluate(x_data, y_data, verbose=0)
print("Mean absolute error :", scores[0])
print("Accuracy :", scores[1])

In [ ]:
# Resultados [SDC, HANG] que obtiene la red para las entradas dadas
y_predict = model.predict(x_data)

# Error absoluto de las predicciones 
errores = np.abs(y_data-model.predict(x_data))

# Error medio absoluto obtenido en las predicciones 
mean_error = np.abs(y_data-model.predict(x_data)).mean(axis=0)
print('Error medio absoluto:')
print('SDC -->', mean_error[0])
print('HANG -->', mean_error[1])

In [ ]:
# Caulcular los resultados para UnAce
unAce_predict = 1 - (y_predict[:,0] + y_predict[:,1])
unAce_real = 1 - (y_data[:,0] + y_data[:,1])

unAce_error = np.abs(unAce_real - unAce_predict)
unAce_MAE = np.abs(unAce_real - unAce_predict).mean(axis=0)

print('Error medio absoluto:')
print('UnAce -->', unAce_MAE)

In [ ]:
results_path = '/content/drive/Shareddrives/NN4FT_DB/Exp_Adrian/Resultados/'
csv = 'bubblesort_RA_TEST.csv'
with open(results_path + csv, "w") as f:
  f.write('SDC_Real;HANG_Real;UnAce_Real;SDC_Predicho;HANG_Predicho;UnAce_Predicho;error_SDC;error_HANG;error_UnAce\n')
  f.close()

string = ""

for i in range(y_data.shape[0]):
  string += (str(y_data[i,0]) + ";" + str(y_data[i,1]) + ";" + str(unAce_real[i]) + ";"
            + str(y_predict[i,0]) + ";" + str(y_predict[i,1]) + ";" + str(unAce_predict[i]) + ";"
            + str(errores[i,0]) + ";" + str(errores[i,1]) + ";" + str(unAce_error[i]) + "\n")
  
  
with open(results_path + csv, "a") as f:
  f.write(string)
  f.close()

In [ ]:
dibujar_Graficas(y_predict, y_data, errores)

## RED NUEVA


# DECLARACIÓN DE MODELOS

## MODELO 1

In [ ]:
def nn_model():
  visible = Input(shape=(x_data.shape[1],))
  
  hidden = Dense(128, activation='relu', kernel_initializer='he_normal') (visible)
  hidden = Dense(128, activation='relu', kernel_initializer='he_normal') (hidden)
  hidden = Dense(22, activation='relu', kernel_initializer='he_normal') (hidden)
  hidden = Dense(22, activation='relu', kernel_initializer='he_normal') (hidden)
  hidden = Dense(64, activation='relu', kernel_initializer='he_normal') (hidden)
  hidden = Dense(44, activation='relu', kernel_initializer='he_normal') (hidden)
  
  output = Dense(2, activation='tanh', kernel_initializer='he_normal') (hidden)
  
  model = Model(visible, output)
  model.compile(optimizer='Adamax', loss='mean_absolute_error', metrics=['accuracy'])
  
  return model

## MODELO 2

In [ ]:
#Define your model
def nn_model2():
  visible = Input(shape=(x_data.shape[1]))
  
  hidden = Dense(128, activation='relu', kernel_initializer='he_normal', kernel_constraint=maxnorm(3)) (visible)
  hidden = Dropout(0.1)(hidden)
  hidden = Dense(128, activation='relu', kernel_initializer='he_normal', kernel_constraint=maxnorm(3)) (hidden)
  hidden = Dropout(0.1)(hidden)
  hidden = Dense(22, activation='relu', kernel_initializer='he_normal', kernel_constraint=maxnorm(3)) (hidden)
  hidden = Dropout(0.1)(hidden)
  hidden = Dense(22, activation='relu', kernel_initializer='he_normal', kernel_constraint=maxnorm(3)) (hidden)
  hidden = Dropout(0.1)(hidden)
  hidden = Dense(64, activation='relu', kernel_initializer='he_normal', kernel_constraint=maxnorm(3)) (hidden)
  hidden = Dropout(0.1)(hidden)
  hidden = Dense(44, activation='relu', kernel_initializer='he_normal', kernel_constraint=maxnorm(3)) (hidden)
  hidden = Dropout(0.1)(hidden)
  
  output = Dense(2, activation='tanh', kernel_initializer='he_normal') (hidden)
  
  model = Model(visible, output)
  model.compile(optimizer=optimizers.Adamax(), loss='mean_absolute_error', metrics=['accuracy'])
  
  return model

# FUNCIONES PARA GRAFICAR

In [ ]:
image_path = '/content/drive/Shareddrives/NN4FT_DB/Exp_Adrian/Graficas/'

In [ ]:
def dibujar_Graficas(predicho, real, errores): 
  
  figura_1 = plt.figure()
  plt.title('SDC (Real vs Predicho)')
  plt.plot(real[:,0],'r+')
  plt.plot(predicho[:,0], 'b+')
  plt.legend(['Real', 'Predicho'], loc='upper left')
  plt.show()
  figura_1.savefig(image_path + "SDC_RP.png")
  plt.clf
  print('\n')
  
  figura_2 = plt.figure()
  plt.title('HANG (Real vs Predicho)')
  plt.plot(real[:,1],'r+')
  plt.plot(predicho[:,1], 'b+')
  plt.legend(['Real', 'Predicho'], loc='upper left')
  plt.show()
  figura_2.savefig(image_path + "HANG_RP.png")
  plt.clf
  print('\n')

  figura_3 = plt.figure()
  plt.title('SDC y HANG (Real)')
  plt.plot(real[:,0],'r+')
  plt.plot(real[:,1], 'b+')
  plt.legend(['SDC', 'HANG'], loc='upper left')
  plt.show()
  figura_3.savefig(image_path + "SDCyHANG_R.png")
  plt.clf
  print('\n')

  figura_4 = plt.figure()
  plt.title('SDC y HANG (Predicho)')
  plt.plot(predicho[:,0],'r+')
  plt.plot(predicho[:,1], 'b+')
  plt.legend(['SDC', 'HANG'], loc='upper left')
  plt.show()
  figura_4.savefig(image_path + "SDCyHANG_P.png")
  plt.clf
  print('\n')

  figura_5 = plt.figure()
  plt.title('Errores absoultos')
  plt.plot(errores[:,0],'r+')
  plt.plot(errores[:,1], 'b+')
  plt.legend(['SDC', 'HANG'], loc='upper left')
  plt.show()
  figura_5.savefig(image_path + "Errores.png")
  plt.clf
  print('\n')


In [ ]:
def graficas_UnAce(predicho, real, errores, unAce_r, unAce_p, unAce_e):

  figura_6 = plt.figure()
  plt.title('UnAce (Real vs Predicho)')
  plt.plot(unAce_r,'r+')
  plt.plot(unAce_p, 'b+')
  plt.legend(['Real', 'Predicho'], loc='upper left')
  plt.show()
  figura_6.savefig(image_path + "unAce_R.png")
  plt.clf
  print('\n')

  figura_7 = plt.figure()
  plt.title('SDC HANG y UnAce (Real)')
  plt.plot(real[:,0],'r+')
  plt.plot(real[:,1], 'b+')
  plt.plot(unAce_r, 'g+')
  plt.legend(['SDC', 'HANG', 'UnAce'], loc='upper left')
  plt.show()
  figura_7.savefig(image_path + "SDCyHANGyUnAce_R.png")
  plt.clf
  print('\n')

  figura_8 = plt.figure()
  plt.title('SDC HANG y UnAce (Predicho)')
  plt.plot(predicho[:,0],'r+')
  plt.plot(predicho[:,1], 'b+')
  plt.plot(unAce_p, 'g+')
  plt.legend(['SDC', 'HANG', 'UnAce'], loc='upper left')
  plt.show()
  figura_8.savefig(image_path + "SDCyHANGyUnAce_P.png")
  plt.clf
  print('\n')

  figura_9 = plt.figure()
  plt.title('Errores absoultos')
  plt.plot(errores[:,0],'r+')
  plt.plot(errores[:,1], 'b+')
  plt.plot(unAce_e, 'g+')
  plt.legend(['SDC', 'HANG', 'UnAce'], loc='upper left')
  plt.show()
  figura_9.savefig(image_path + "Errores_con_UnAce.png")
  plt.clf
  print('\n')
  